In [1]:
import os
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split

from data_fixing import format_and_label_data, format_image
from train_model import create_CNN_model,CNN_fit_train,CNN_accuracy,load_CNN_model,CNN_one_pred


Resize and greyscale images:

In [2]:
images_labs = format_and_label_data('../images/dataset', 128, False)

image_data = images_labs[0]
labels = images_labs[1]


Skipping 00000114.png, invalid shape: (128, 128)
Skipping 00000190.png, invalid shape: (128, 128)
Skipping 00000009.png, invalid shape: (128, 128)
Skipping 00000021.png, invalid shape: (128, 128)
Skipping 00000061.png, invalid shape: (128, 128)
Skipping 00000078.png, invalid shape: (128, 128)
Skipping 00000172.png, invalid shape: (128, 128)
Skipping 59bef3942b6041b3a6e0526100264536.jpg, invalid shape: (128, 128)


Display a image to check it worked

In [3]:
img = image_data[0].reshape(128, 128,3)
img_pillow = Image.fromarray(img)
img_pillow.show()

print(image_data.shape)
print(labels.shape)

(847, 128, 128, 3)
(847,)


Split data

In [4]:
X_train, X_test, y_train, y_test = train_test_split(image_data, labels, test_size=0.3, random_state=420)

Model:

In [ ]:


model = create_CNN_model(dropout=0.45,grayscale=False,image_size=128)

model = CNN_fit_train(model, X_train,y_train,X_test,y_test,75,32, './models/best_model_func.keras')
# model.summary()

d:\anaconda\envs\DrawEmAll\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/75
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step - accuracy: 0.4671 - loss: 2.7727
Epoch 1: val_accuracy improved from -inf to 0.62353, saving model to ./models/best_model_func.keras
19/19 ━━━━━━━━━━━━━━━━━━━━ 12s 278ms/step - accuracy: 0.4706 - loss: 2.7193 - val_accuracy: 0.6235 - val_loss: 0.6091
Epoch 2/75
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 225ms/step - accuracy: 0.7438 - loss: 0.5653
Epoch 2: val_accuracy improved from 0.62353 to 0.93333, saving model to ./models/best_model_func.keras
19/19 ━━━━━━━━━━━━━━━━━━━━ 5s 253ms/step - accuracy: 0.7467 - loss: 0.5598 - val_accuracy: 0.9333 - val_loss: 0.2717
Epoch 3/75
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step - accuracy: 0.9078 - loss: 0.2606
Epoch 3: val_accuracy improved from 0.93333 to 0.96078, saving model to ./models/best_model_func.keras
19/19 ━━━━━━━━━━━━━━━━━━━━ 5s 253ms/step - accuracy: 0.9087 - loss: 0.2583 - val_accuracy: 0.9608 - val_loss: 0.2427
Epoch 4/75
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step - accuracy: 0.9292 - loss: 0.170

test on hand drawn images

In [7]:
hand_data_and_lab = format_and_label_data('../images/Hand_Drawn')
hand_data = hand_data_and_lab[0]
labels = hand_data_and_lab[1]

In [11]:
img = hand_data[0].reshape(128,128 ,3)
img_pillow = Image.fromarray(img)
img_pillow.show()

In [ ]:


model  = load_CNN_model('./models/best_model_func.keras')
acc = CNN_accuracy(model, hand_data, labels)
print(acc) 
#note accuracy is low because of the non-colored drawings being performed poorly on

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
0.42857142857142855


Note: 30 epochs for all the colored ones to be guessed correctly..., non colored defaults to squirtle

Using data gen now:

In [5]:
model = create_CNN_model(dropout=0.45,grayscale=False,image_size=128)

model = CNN_fit_train(model, X_train,y_train,X_test,y_test,75,32, './models/best_model_func.keras', datagen = True)
# model.summary()

d:\anaconda\envs\DrawEmAll\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/75


d:\anaconda\envs\DrawEmAll\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 292ms/step - accuracy: 0.4494 - loss: 2.6105
Epoch 1: val_accuracy improved from -inf to 0.80392, saving model to ./models/best_model_func.keras
19/19 ━━━━━━━━━━━━━━━━━━━━ 11s 336ms/step - accuracy: 0.4551 - loss: 2.5575 - val_accuracy: 0.8039 - val_loss: 0.4196
Epoch 2/75
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 261ms/step - accuracy: 0.8530 - loss: 0.4975
Epoch 2: val_accuracy did not improve from 0.80392
19/19 ━━━━━━━━━━━━━━━━━━━━ 6s 282ms/step - accuracy: 0.8533 - loss: 0.5023 - val_accuracy: 0.7843 - val_loss: 0.5358
Epoch 3/75
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 282ms/step - accuracy: 0.8768 - loss: 0.3729
Epoch 3: val_accuracy improved from 0.80392 to 0.81569, saving model to ./models/best_model_func.keras
19/19 ━━━━━━━━━━━━━━━━━━━━ 12s 629ms/step - accuracy: 0.8777 - loss: 0.3710 - val_accuracy: 0.8157 - val_loss: 0.5138
Epoch 4/75
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 290ms/step - accuracy: 0.9123 - loss: 0.2393
Epoch 4: val_accuracy improved from 0.81569 to 0.87843, savi

In [8]:
model  = load_CNN_model('./models/best_model_func.keras')
acc = CNN_accuracy(model, hand_data, labels)
print(acc) 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
0.7142857142857143


Black and White:

Greyscale model

In [9]:

data = format_and_label_data(grayscale=True)

image_date = data[0]
labels = data[1]



Skipping (001) Bulbasaur (Screenshot).jfif, invalid shape: (128, 128)
Skipping 00000000.png, invalid shape: (128, 128)
Skipping 00000001.png, invalid shape: (128, 128)
Skipping 00000002.png, invalid shape: (128, 128)
Skipping 00000003.png, invalid shape: (128, 128)
Skipping 00000004.png, invalid shape: (128, 128)
Skipping 00000005.png, invalid shape: (128, 128)
Skipping 00000006.jpg, invalid shape: (128, 128)
Skipping 00000007.jpg, invalid shape: (128, 128)
Skipping 00000008.png, invalid shape: (128, 128)
Skipping 00000009.png, invalid shape: (128, 128)
Skipping 00000010.png, invalid shape: (128, 128)
Skipping 00000011.png, invalid shape: (128, 128)
Skipping 00000012.png, invalid shape: (128, 128)
Skipping 00000013.png, invalid shape: (128, 128)
Skipping 00000014.png, invalid shape: (128, 128)
Skipping 00000015.jpg, invalid shape: (128, 128)
Skipping 00000016.png, invalid shape: (128, 128)
Skipping 00000017.png, invalid shape: (128, 128)
Skipping 00000018.jpg, invalid shape: (128, 128)

d:\anaconda\envs\DrawEmAll\lib\site-packages\PIL\Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Skipping 00000009.png, invalid shape: (128, 128)
Skipping 00000010.png, invalid shape: (128, 128)
Skipping 00000011.png, invalid shape: (128, 128)
Skipping 00000012.jpg, invalid shape: (128, 128)
Skipping 00000013.jpg, invalid shape: (128, 128)
Skipping 00000014.jpg, invalid shape: (128, 128)
Skipping 00000015.png, invalid shape: (128, 128)
Skipping 00000016.png, invalid shape: (128, 128)
Skipping 00000017.png, invalid shape: (128, 128)
Skipping 00000018.png, invalid shape: (128, 128)
Skipping 00000019.png, invalid shape: (128, 128)
Skipping 00000020.png, invalid shape: (128, 128)
Skipping 00000021.png, invalid shape: (128, 128)
Skipping 00000022.png, invalid shape: (128, 128)
Skipping 00000023.jpg, invalid shape: (128, 128)
Skipping 00000024.jpg, invalid shape: (128, 128)
Skipping 00000025.png, invalid shape: (128, 128)
Skipping 00000026.png, invalid shape: (128, 128)
Skipping 00000027.png, invalid shape: (128, 128)
Skipping 00000028.png, invalid shape: (128, 128)
Skipping 00000029.pn

KeyboardInterrupt: 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(image_data, labels, test_size=0.3, random_state=410)

In [ ]:
model = create_CNN_model(dropout=0.45,grayscale=True,image_size=128)

model = CNN_fit_train(model, X_train,y_train,X_test,y_test,75,32, './models/best_model_funcbw.keras', datagen = True)

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_17 (Conv2D)              │ (None, 126, 126, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 126, 126, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 28, 28, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │       802,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 858,947 (3.28 MB)

 Trainable params: 858,883 (3.28 MB)

 Non-trainable params: 64 (256.00 B)

In [2]:
model  = load_CNN_model('./models/best_model_func.keras')
# acc = CNN_accuracy(model, hand_data, labels)
# print(acc) 

In [ ]:
img = format_image(image_path = '../images/Hand_Drawn/Bulbasaur/bulb_color.png', grayscale = False, image_size = 128)
print(img.shape)
print(CNN_one_pred(model, img))


(128, 128, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
Bulbasaur
